

# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('../output_data/cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng",
                                      "Lat",
                                   geo=True,
                                 alpha=0.55,
                          size = "Humidity",
                              tiles = "OSM",
                          frame_width = 700,frame_height = 500)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
my_cities_data_df=city_data_df.loc[(city_data_df["Max Temp"]<33)
                                   & (city_data_df["Max Temp"]>29)
                                   & (city_data_df["Cloudiness"]<25)
                                   & (city_data_df["Wind Speed"]>5.0)
                                   & (city_data_df["Humidity"]<90)]

# Drop any rows with null values
my_cities_data_df.dropna()

# Display sample data
my_cities_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,98,itarema,-2.9248,-39.9167,31.07,54,14,9.48,BR,1666108293
286,286,praia,14.9215,-23.5087,29.30,74,20,8.75,CV,1666108277
300,300,aquiraz,-3.9014,-38.3911,31.92,56,22,7.50,BR,1666108415
466,466,the valley,18.2170,-63.0578,30.05,65,20,6.17,AI,1666108507
479,479,praya,14.9215,-23.5087,29.30,74,20,8.75,CV,1666108514


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=my_cities_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
98,itarema,BR,-2.9248,-39.9167,54,
286,praia,CV,14.9215,-23.5087,74,
300,aquiraz,BR,-3.9014,-38.3911,56,
466,the valley,AI,18.2170,-63.0578,65,
479,praya,CV,14.9215,-23.5087,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 100000
params = {"categories": "accommodation.hotel",
              "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
itarema - nearest hotel: Pousada Oásis
praia - nearest hotel: Casa Sodadi
aquiraz - nearest hotel: Pousada tia Fabíola
the valley - nearest hotel: CeBlue
praya - nearest hotel: Casa Sodadi


,City,Country,Lat,Lng,Humidity,Hotel Name
98,itarema,BR,-2.9248,-39.9167,54,Pousada Oásis
286,praia,CV,14.9215,-23.5087,74,Casa Sodadi
300,aquiraz,BR,-3.9014,-38.3911,56,Pousada tia Fabíola
466,the valley,AI,18.2170,-63.0578,65,CeBlue
479,praya,CV,14.9215,-23.5087,74,Casa Sodadi


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points("Lng",
                                        "Lat",
                                     geo=True,
                                      alpha=1,
                            size = "Humidity",
                                tiles = "OSM",
          hover_cols=["Hotel Name","Country"],
                            frame_width = 700,
                           frame_height = 700)

# Display the map
city_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)